In [6]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from IPython.display import display
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
from numpy import nan
display(HTML("<style>"
    + "#notebook { padding-top:0px; } " ""
    + ".container { width:100%; } "
    + ".end_space { min-height:0px; } "
    + "</style>"))
# import colors


def highlight_max(sett):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    is_max = s == s.max()
    
    return ['font-weight: bold' if v else '' for v in is_max]

def highlight_group(sett):
    try:
        if sett.dtype == 'O':
            oboe = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
        else:
            oboe = sett
        is_max = oboe.to_frame().apply(lambda x: oboe[oboe.index.get_level_values(0) == x.name[0]].max() == x.item(), axis=1)
        return ['font-weight: bold' if v else '' for v in is_max]
    except:
        return ['' for _ in sett]

def highlight_group_min(sett):
    try:
        if sett.dtype == 'O':
            oboe = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
        else:
            oboe = sett
        is_max = oboe.to_frame().apply(lambda x: oboe[oboe.index.get_level_values(0) == x.name[0]].min() == x.item(), axis=1)
        return ['font-weight: bold' if v else '' for v in is_max]
    except:
        return ['' for _ in sett]

    
def highlight_min(sett):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    is_max = s == s.min()
    
    return ['font-weight: bold' if v else '' for v in is_max]

def b_g(sett, cmap='PuBu', low=0, high=0):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    a = s
    rng = a.max() - a.min()
    norm = colors.Normalize(a.min() - (rng * low),
                        a.max() + (rng * high))
    normed = norm(a.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

def method_to_df(apath, setting, dataset, method):
    last = ' '.join([setting, dataset, method])
    with open(apath + '%s/%s/' % (setting, dataset) + method +'/logs.pyd', 'r') as f:
        lines = f.read().splitlines()
    o = pd.DataFrame.from_records([eval(x.replace('nan', 'float(\'nan\')')) for x in lines])
    
    o = o.rename(columns={'cs_size':'buffer_size'})
    if 'buffer_size' not in o:
        o['buffer_size'] = 0
    if 'job_number' in o.columns:
        del o['job_number']
    o['method'] = method
    return o


req_rep = 1
print_args= False
display_pre = False
howmany_pre = 1
print_other_metrics = False
t_distinct_cols = ['buffer_size', 'method', 'shap_weight', 'shap_all', 'n_epochs']
ignore_cols = []
excluded_methods = []
included_methods = []
excluded_datasets = []

sec_respath = None

# PAMI
respath = None #'../storage/results/dataset-can-mnist/classes_per_task-1/logs.pyd'#'C:\\Users\\emace\\AImageLab\\SRV-Continual\\results\\mammoth\\results\\'
sec_respath = None #'../storage/results/dataset-can-mnist/classes_per_task-2/logs.pyd' #'/home/mbosc/srv-nas/mammoth-master/results/'
third_respath = '../storage/results/dataset-can-sole/classes_per_task-1/logs.pyd'
# third_respath = None #'/home/mbosc/phd/pami/results_buzz/'

respaths = [respath, sec_respath, third_respath]
islist = lambda col: any([type(x) == list for x in col.values])


In [9]:
## import file
list_total_parsed = []
for path in respaths:
    if path is not None:
        with open(os.path.join(path), 'r') as obj:
            file = obj.readlines()
        file_parsed = map(eval, file)
        list_total_parsed.extend(list(file_parsed))
    
df_file = pd.DataFrame(list_total_parsed)
df_file['timestamp'] = pd.to_datetime(df_file['timestamp'])
# df_file = df_file[df_file['timestamp']>'2022-02-02 16:50:00']
df_file = df_file[df_file['timestamp']>'2022-02-05 16:50:00']
# df_file.shape

In [11]:
# total_auc = pd.concat([pd.DataFrame.from_dict(sing[1].loc[0,'auc_per_task'], orient='index')]*2)
# total_auc.reset_index().groupby('index').mean()

def calc_mean_on_matrix(X):
    total_df = pd.concat([pd.DataFrame.from_dict(x, orient='index') for x in X])
    return total_df.reset_index().groupby('index').mean().to_dict(orient='index')

In [12]:
def calc_mean_pm_std(x):
    return f'{x.mean().round(3)}+-{x.std().round(3)}'

In [13]:
# Converting links to html tags
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [21]:
df_elab = df_file.copy()

common_keys=['dataset','classes_per_task','model','optim','lr','batch_size','n_epochs','latent_space','joint']
part_keys = {'sae':['sparse_weight','norm_order'], 'ae':[], 'vae':['beta_kl'],'dae':['noise_mean','noise_std']}
elaboration_cols = {
    'auc_final':[calc_mean_pm_std, len],
    'auc_average':[calc_mean_pm_std, len],
    'auc_per_task': [calc_mean_on_matrix],
    'conf_matrix_per_task': [calc_mean_on_matrix]
    
}
task_per_task_elaboration_cols = {
    
}
total_list = []
for model, df_model in df_elab.groupby('model'):
    df_model['model'] = model
    total_keys = common_keys+part_keys[model]
    
    df_single = df_model.groupby(total_keys).agg(elaboration_cols)
        
    total_list.append(df_single.reset_index())

df_total = pd.concat(total_list).set_index(common_keys)

In [17]:
df_single

auc_final          auc_average                                           auc_per_task                               conf_matrix_per_task
                                                                                           calc_mean_pm_std len calc_mean_pm_std len                                calc_mean_on_matrix                                calc_mean_on_matrix
dataset  classes_per_task model optim lr    batch_size n_epochs latent_space joint beta_kl                                                                                                                                                
can-sole 1                vae   adam  0.010 64         50       10           True  0.5           0.472+-nan   1       0.472+-nan   1  {'0': {'[0]': 0.47185630461963896, 'total': 0....  {'0[0]': {0.0: 2.604145967822843, 1.0: 2.48574...
                                sgd   0.001 64         50       8            True  0.2           0.419+-nan   1       0.419+-nan   1  {'0': {'[0]': 0.41926442549086207, 'total': 0....  {'0[0]': {0.0: 5.001989004531414, 1.0: 4.53712...
                                                                                   0.5           0.411+-nan   1       0.411+-nan   1  {'0': {'[0]': 0.4110563159961656, 'total': 0.4...  {'0[0]': {0.0: 4.080044531410413, 1.0: 3.60115...
                                                                                   0.7           0.437+-nan   1       0.437+-nan   1  {'0': {'[0]': 0.43676937263923693, 'total': 0....  {'0[0]': {0.0: 3.512057158466667, 1.0: 3.23811...
                                                                                   1.0           0.431+-nan   1       0.431+-nan   1  {'0': {'[0]': 0.431168402133977, 'total': 0.43...  {'0[0]': {0.0: 3.61149031918757, 1.0: 3.303745...
                                                                16           True  0.2           0.429+-nan   1       0.429+-nan   1  {'0': {'[0]': 0.42859695152834104, 'total': 0....  {'0[0]': {0.0: 3.583528842008661, 1.0: 3.26750...
                                                                                   0.5           0.441+-nan   1       0.441+-nan   1  {'0': {'[0]': 0.4405721408545334, 'total': 0.4...  {'0[0]': {0.0: 3.2697421465777294, 1.0: 2.9935...
                                                                                   0.7           0.431+-nan   1       0.431+-nan   1  {'0': {'[0]': 0.43124606880830085, 'total': 0....  {'0[0]': {0.0: 3.2494540158255316, 1.0: 2.8919...
                                                                                   1.0           0.415+-nan   1       0.415+-nan   1  {'0': {'[0]': 0.41506626826944737, 'total': 0....  {'0[0]': {0.0: 3.693874972625868, 1.0: 3.23692...
                                      0.010 64         50       8            True  0.2           0.457+-nan   1       0.457+-nan   1  {'0': {'[0]': 0.45675123036045334, 'total': 0....  {'0[0]': {0.0: 2.968303412710003, 1.0: 2.77832...
                                                                                   0.5           0.472+-nan   1       0.472+-nan   1  {'0': {'[0]': 0.47232011687193964, 'total': 0....  {'0[0]': {0.0: 2.6465928581586944, 1.0: 2.5307...
                                                                                   0.7           0.421+-nan   1       0.421+-nan   1  {'0': {'[0]': 0.4211867030278002, 'total': 0.4...  {'0[0]': {0.0: 3.8427684706717025, 1.0: 3.4531...
                                                                                   1.0           0.466+-nan   1       0.466+-nan   1  {'0': {'[0]': 0.4663143225004764, 'total': 0.4...  {'0[0]': {0.0: 2.5934613673105664, 1.0: 2.4491...
                                                                16           True  0.2           0.454+-nan   1       0.454+-nan   1  {'0': {'[0]': 0.4537232692637998, 'total': 0.4...  {'0[0]': {0.0: 2.9561705321479312, 1.0: 2.7630...
                                                                                   0.5           0.473+

In [22]:
def plot_heatmap(matrix_dict, title=None, cmap='Reds'):
    fig, ax = plt.subplots(figsize=(10,8))
    sns.heatmap(pd.DataFrame.from_dict(matrix_dict, orient='index'), annot=True, cbar=False, cmap=cmap, ax=ax, linewidths=0.1)
    ax.set_xlabel('Label')
    ax.set_ylabel('Task')
    if title is not None:
        ax.set_title(title)
def plot_heatmaps(df: pd.DataFrame, which_col=('auc_per_task', 'calc_mean_on_matrix')):
    for title,row in df.iterrows():
        plot_heatmap(row.loc[which_col], title='\n'.join(f'{x}: {y}' for x,y in zip(df.index.names,title)))

In [31]:
df_total

auc_average            auc_final                                           auc_per_task beta_kl                               conf_matrix_per_task noise_mean noise_std norm_order sparse_weight
                                                                                   calc_mean_pm_std len calc_mean_pm_std len                                calc_mean_on_matrix                                        calc_mean_on_matrix                                              
dataset  classes_per_task model optim lr    batch_size n_epochs latent_space joint                                                                                                                                                                                                      
can-sole 1                ae    sgd   0.001 64         50       8            True         0.47+-nan   1        0.47+-nan   1  {'0': {'[0]': 0.4704394347616378, 'total': 0.4...     NaN  {'0[0]': {0.0: 2.827783498512905, 1.0: 2.69091...        NaN       NaN        NaN           NaN
                                                                16           True        0.446+-nan   1       0.446+-nan   1  {'0': {'[0]': 0.44605166146519076, 'total': 0....     NaN  {'0[0]': {0.0: 3.357665805235332, 1.0: 3.08126...        NaN       NaN        NaN           NaN
                                      0.010 64         50       8            True        0.406+-nan   1       0.406+-nan   1  {'0': {'[0]': 0.40641496647753494, 'total': 0....     NaN  {'0[0]': {0.0: 1.332090754720861, 1.0: 1.13361...        NaN       NaN        NaN           NaN
                                                                16           True        0.451+-nan   1       0.451+-nan   1  {'0': {'[0]': 0.4505604580364799, 'total': 0.4...     NaN  {'0[0]': {0.0: 1.6399177560020113, 1.0: 1.4058...        NaN       NaN        NaN           NaN
                                      0.100 64         50       8            True         0.48+-nan   1        0.48+-nan   1  {'0': {'[0]': 0.48022174382468213, 'total': 0....     NaN  {'0[0]': {0.0: 0.7473392997568016, 1.0: 0.7046...        NaN       NaN        NaN           NaN
                                                                16           True        0.467+-nan   1       0.467+-nan   1  {'0': {'[0]': 0.4674803891647364, 'total': 0.4...     NaN  {'0[0]': {0.0: 0.6860104249640022, 1.0: 0.6401...        NaN       NaN        NaN           NaN
                          dae   sgd   0.001 64         50       8            True        0.429+-nan   1       0.429+-nan   1  {'0': {'[0]': 0.4294213942152581, 'total': 0.4...     NaN  {'0[0]': {0.0: 2.30576910099566, 1.0: 2.069182...        0.0       0.5        NaN           NaN
                                      0.010 64         50       8            True        0.452+-nan   1       0.452+-nan   1  {'0': {'[0]': 0.452140537300244, 'total': 0.45...     NaN  {'0[0]': {0.0: 1.7081262359937603, 1.0: 1.5750...        0.0       0.5        NaN           NaN
                                                                16           True        0.443+-nan   1       0.443+-nan   1  {'0': {'[0]': 0.44295954331995824, 'total': 0....     NaN  {'0[0]': {0.0: 1.6061191670206787, 1.0: 1.4382...        0.0       0.5        NaN           NaN
                                      0.100 64         50       8            True        0.485+-nan   1       0.485+-nan   1  {'0': {'[0]': 0.48473070448049516, 'total': 0....     NaN  {'0[0]': {0.0: 0.7833103308443824, 1.0: 0.7513...        0.0       0.5        NaN           NaN
                                                                16           True        0.467+-nan   1       0.467+-nan   1  {'0': {'[0]': 0.4674544091152266, 'total': 0.4...     NaN  {'0[0]': {0.0: 0.8774253210438903, 1.0: 0.8209...        0.0       0.5        NaN           NaN
                          sae   sgd   0.001 64         50       8            True        0.439+-nan   1       0.439+-nan   1  {'0': {'[0]': 0.439036911360

In [ ]:
plot_heatmaps(df_total)

In [ ]:
sns.heatmap(df_total.iloc[-2, -2])

In [ ]:
# gridmap(methods, ['3ea70747-8697-4e76-bc4f-319c6fa312b5', '89f4c672-f16a-4b08-a509-a94c82a6ebbd', '31d4cc8c-a727-4665-8643-2212eb519aa6', '26ea2435-d2d4-4988-b37e-d95934301d2f', 'a0626b3b-b34f-4353-8453-4abd769c130c', '9ae21984-b77f-4d12-be42-c4929c4d4125', '1cc0e452-8112-47ac-b939-f9ad5ae5edc6', '86d1bc60-d263-46be-b90f-ff9d9a30173a', '606653f0-b606-426c-9e8f-7770e8dad36a', '6ef54be8-cedf-40d7-a7c7-cda396b45d1c', '918ff2d4-b0d1-456d-96a1-e283e7838827'],
# ['57976485-efee-4aae-95e2-4a2ac112ac3e', 'ff4eb4f9-0e6b-4336-bd66-057a02d2f663', '590d6225-04f7-4ba6-b8c7-0182262ccdf5', '78b3b890-bcc0-4fa6-bc7a-2569f12cee23', '0c875241-6230-41d1-9875-09a2c4c586c0', 'f608773d-99e2-49c9-b1d2-6857966d6977', 'f9a60776-ec0b-47fd-8895-27743c5c9911', '74b41430-89b3-4c1c-a058-9e01bb05480a', '45679ee3-d077-4e5a-83cc-77d4670d27ea', '1e7573db-8821-4cd7-b243-3f4fe9d31553', 'ad3fa7bc-8a5a-4fa6-a2b1-5d3072597e02'])